In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from pathlib import Path

In [2]:
from Model import ConvNet, train, test, print_size_of_model

In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler
_ = torch.manual_seed(0)

transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Assuming dataset_folder is your folder with all data.
dataset = ImageFolder(root='./data/archive_weather/Multi-class Weather Dataset/', transform=transform)

# Get the size of the dataset.
dataset_size = len(dataset)

# Calculate the lengths of each part.
train_size = int(0.7 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

# Use random_split to split the dataset.
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create the DataLoaders.
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_size, shuffle=True)

device = 'cpu'

In [4]:
net = ConvNet(output_size=4).to(device)

In [5]:
MODEL_FILENAME = './Saved_models/Weather.pt'

if Path(MODEL_FILENAME).exists():
    net.load_state_dict(torch.load(MODEL_FILENAME))
    print('Loaded model from disk')
else:
    train(net, train_loader, val_loader, 1000, 5, device=device)
    torch.save(net.state_dict(), MODEL_FILENAME)

Loaded model from disk


In [6]:
print(f'Accuracy of the model before quantization: {test(net, test_loader)}')

  0%|          | 0/1 [00:00<?, ?it/s]

Accuracy of the model before quantization: 94.78


In [7]:
from Factorization import factorize, SVD_quant

In [8]:
reduction_rate = 2

In [9]:
import re

net_state_copy = net.state_dict().copy()
linear_weights_keys = [layer for layer in net_state_copy if 'linear' in layer and '.weight' in layer]

for layer in linear_weights_keys[1:-1]:
    print(layer)
    W = net.state_dict()[layer].detach()
    # net_state_copy[layer] = factorize(W, torch.linalg.matrix_rank(W) // reduction_rate, method='min-max', use_ADMM=False)
    net_state_copy[layer] = SVD_quant(W, torch.linalg.matrix_rank(W) // reduction_rate, method='min-max')

linear2.weight


In [10]:
new_net = ConvNet(output_size=4).to(device)
new_net.load_state_dict(net_state_copy)

test(new_net, test_loader)

  0%|          | 0/1 [00:00<?, ?it/s]

93.91

In [11]:
import re

net_state_copy = net.state_dict().copy()
linear_weights_keys = [layer for layer in net_state_copy if 'linear' in layer and '.weight' in layer]

mem_usage_init = []
mem_usage_compressed = []

for layer in linear_weights_keys[1:-1]:
    W = net.state_dict()[layer].detach()
    A, B, sc = factorize(W, torch.linalg.matrix_rank(W) // reduction_rate, return_int=True)

    mem_usage_init.append(W.element_size() * W.numel())
    mem_usage_compressed.append(A.element_size() * A.numel() + B.element_size() * B.numel())

print(f'Initial (byte): {sum(mem_usage_init)}')
print(f'Compressed (byte): {sum(mem_usage_compressed)}')

rank: 32
Initial (byte): 131072
Compressed (byte): 18432
